In [2]:
import pandas as pd
import torch
import sys
import numpy as np
from scipy.signal import spectrogram
sys.path.append('../src')
import matplotlib.pyplot as plt
from glob import glob
import os


In [3]:
def format_gearbox(file_name, segment_length, nperseg):
    X = []
    y = []

    if "20_0" in file_name:
        domain = 0 
        frequency = 20

    elif "30_2" in file_name:
        domain = 1
        frequency = 30
    
    if "ball" in file_name and frequency==20 and domain==0:
        sep = ","
    else:
        sep = "\t"
    
    df = pd.read_csv(file_name, skiprows=16, header=None, usecols=range(8), sep=sep)
    cols = ["ch1", "ch2", "ch3", "ch4", "ch5", "ch6", "ch7", "ch8"]
    df.columns = cols

    if "ball" in file_name:
        df["label"] = 0

    elif "comb" in file_name:
        df["label"] = 1

    elif "inner" in file_name:
        df["label"] = 2

    elif "outer" in file_name:
        df["label"] = 3

    elif "health" in file_name:
        df["label"] = 4

    N = df.shape[0]
    val = df[cols].values
    splitted_val = np.stack(np.array_split(val[:int(N//segment_length * segment_length)], N//segment_length))
    for i in range(splitted_val.shape[0]):
        temp_X = []
        for j in range(splitted_val.shape[2]):
            f, t, sxx = spectrogram(splitted_val[i, :, j], fs=frequency, nperseg=nperseg)
            temp_X.append(sxx)

        y.append(df['label'][0])
        X.append(np.stack(temp_X))

    X = np.stack(X)
    y = np.stack(y)

    return X, y

X, y = format_gearbox("/data/home/jkataok1/DA_DFD/data/raw/gearbox/bearingset/ball_20_0.csv", segment_length=2048*2, nperseg=128)
X.shape, y.shape



((255, 8, 65, 36), (255,))

In [4]:
def prepare_gearbox(data_folder, segment_length, nperseg):
    X_domain0 = []
    y_domain0 = []
    X_domain1 = []
    y_domain1 = []

    files = glob(data_folder + "/*")
    files_domain0 = [file for file in files if "20_0" in file]
    files_domain1 = [file for file in files if "30_2" in file]
    print(files)

    for file in files_domain0:
        print(file)
        _X, _y = format_gearbox(file, segment_length, nperseg)
        X_domain0.append(_X)
        y_domain0.append(_y)
    
    for file in files_domain1:
        print(file)
        _X, _y = format_gearbox(file, segment_length, nperseg)
        X_domain1.append(_X)
        y_domain1.append(_y)

    X_domain0 = np.concatenate(X_domain0)
    X_domain0 = (X_domain0 - X_domain0.mean(axis=(0, -2, -1), keepdims=True)) / (X_domain0.std(axis=(0, -2, -1), keepdims=True)+1e-6)
    y_domain0 = np.concatenate(y_domain0)
    y_domain0 = y_domain0.reshape(-1, 1)

    X_domain1 = np.concatenate(X_domain1)
    X_domain1 = (X_domain1 - X_domain1.mean(axis=(0, -2, -1), keepdims=True)) / (X_domain1.std(axis=(0, -2, -1), keepdims=True)+1e-6)

    y_domain1 = np.concatenate(y_domain1)
    y_domain1 = y_domain1.reshape(-1, 1)

    save_path = "/data/home/jkataok1/DA_DFD/data/processed/gearbox"
    np.savez(os.path.join(save_path, "0_spectrogram.npz"), x=X_domain0, y=y_domain0)
    np.savez(os.path.join(save_path, "1_spectrogram.npz"), x=X_domain1, y=y_domain1)

prepare_gearbox("/data/home/jkataok1/DA_DFD/data/raw/gearbox/bearingset", segment_length=2048, nperseg=128)

['/data/home/jkataok1/DA_DFD/data/raw/gearbox/bearingset/health_30_2.csv', '/data/home/jkataok1/DA_DFD/data/raw/gearbox/bearingset/ball_20_0.csv', '/data/home/jkataok1/DA_DFD/data/raw/gearbox/bearingset/outer_20_0.csv', '/data/home/jkataok1/DA_DFD/data/raw/gearbox/bearingset/outer_30_2.csv', '/data/home/jkataok1/DA_DFD/data/raw/gearbox/bearingset/health_20_0.csv', '/data/home/jkataok1/DA_DFD/data/raw/gearbox/bearingset/ball_30_2.csv', '/data/home/jkataok1/DA_DFD/data/raw/gearbox/bearingset/comb_30_2.csv', '/data/home/jkataok1/DA_DFD/data/raw/gearbox/bearingset/inner_20_0.csv', '/data/home/jkataok1/DA_DFD/data/raw/gearbox/bearingset/comb_20_0.csv', '/data/home/jkataok1/DA_DFD/data/raw/gearbox/bearingset/inner_30_2.csv']
/data/home/jkataok1/DA_DFD/data/raw/gearbox/bearingset/ball_20_0.csv
/data/home/jkataok1/DA_DFD/data/raw/gearbox/bearingset/outer_20_0.csv
/data/home/jkataok1/DA_DFD/data/raw/gearbox/bearingset/health_20_0.csv
/data/home/jkataok1/DA_DFD/data/raw/gearbox/bearingset/inner_

In [14]:
np.load("/data/home/jkataok1/DA_DFD/data/processed/gearbox/1_spectrogram.npz", allow_pickle=True)["x"].shape

(2555, 8, 65, 18)